# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [37]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [38]:
df = pd.read_csv('./data/Orders/Orders.csv') 

In [39]:
df.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [40]:
df.head(3)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00


In [44]:
#sub problem 1
df_agg = df.groupby("CustomerID")["amount_spent"].sum().reset_index()
df_agg.rename(columns={"amount_spent": "total_spent"}, inplace=True)
df_agg

,CustomerID,total_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [ ]:
#sub problem 2
low = df_agg["total_spent"].quantile(0.90)
high =  df_agg["total_spent"].quantile(0.95)
df_selected = df_agg[(df_agg["total_spent"] >= low) & (df_agg["total_spent"] < high)]
df_selected


,CustomerID,total_spent
1,12347,4310.00
15,12362,5226.23
27,12378,4008.62
56,12417,3649.10
67,12429,3750.40
...,...,...
4214,18118,5653.82
4273,18198,5425.56
4294,18225,5509.12
4295,18226,5228.05


In [53]:
#sub problem 3
q90 = df_agg["total_spent"].quantile(0.90)
q95 =  df_agg["total_spent"].quantile(0.95)
def label_customer(x):
    if x >=q95:
        return "VIP"
    elif x>=q90:
        return "preferred"
    else: 
        return "Regular"
df_agg["label"] = df_agg["total_spent"].apply(label_customer)    
df_labeled = df.merge(df_agg[["CustomerID", "label"]], on="CustomerID", how="left")
df_labeled



,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,Regular
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,Regular
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Regular
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [55]:
vip_by_country = (df_labeled[df_labeled["label"] == "VIP"].groupby("Country")["CustomerID"].nunique().sort_values(ascending=False))
vip_by_country

Country
United Kingdom     177
Germany             10
France               9
Switzerland          3
Spain                2
Australia            2
EIRE                 2
Portugal             2
Japan                2
Belgium              1
Finland              1
Channel Islands      1
Cyprus               1
Denmark              1
Singapore            1
Norway               1
Netherlands          1
Sweden               1
Name: CustomerID, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [57]:
vip_pref_by_country = (df_labeled[df_labeled["label"].isin(["VIP", "Preferred"])].groupby("Country")["CustomerID"].nunique().sort_values(ascending=False))
vip_pref_by_country

Country
United Kingdom     177
Germany             10
France               9
Switzerland          3
Spain                2
Australia            2
EIRE                 2
Portugal             2
Japan                2
Belgium              1
Finland              1
Channel Islands      1
Cyprus               1
Denmark              1
Singapore            1
Norway               1
Netherlands          1
Sweden               1
Name: CustomerID, dtype: int64